# **v1.0**

*   Hugging Face API
*   Minimum required agents
*   Model: microsoft/Phi-3-mini-4k
*   Can't work on the cases from data.csv





# Hugging Face login

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

# Creating Agents

## CourtroomAgent Class

In [ ]:
from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient

FORBIDDEN_SECTION_HEADERS = [
    "Instruction:", "Context:", "Example Dialogue:", "Dialogue Role:",
    "Difficulty:", "Behavior:", "Ethics:", "You are now"
]

CRIMINAL_KEYWORDS = [
    "felony", "murder", "assault", "robbery", "homicide", "theft", "manslaughter",
    "arrested", "charged", "criminal", "indicted", "burglary", "prosecuted", "violence", "drug trafficking"
]

CIVIL_KEYWORDS = [
    "contract", "damages", "lawsuit", "breach", "negligence", "compensation", "dispute",
    "settlement", "liable", "plaintiff", "property damage", "alimony", "divorce", "injunction", "sued"
]

class CourtroomAgent:
    case_type = None  # Static class-level var

    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []
        self.client = InferenceClient(
            model,
            token=os.getenv("HF_API_TOKEN")
        )

    @staticmethod
    def deduce_case_type(case_background: str) -> str:
        lower_text = case_background.lower()

        criminal_score = sum(kw in lower_text for kw in CRIMINAL_KEYWORDS)
        civil_score = sum(kw in lower_text for kw in CIVIL_KEYWORDS)

        if criminal_score > civil_score:
            CourtroomAgent.case_type = "criminal"
        elif civil_score > criminal_score:
            CourtroomAgent.case_type = "civil"
        else:
            CourtroomAgent.case_type = "civil"  # Default fallback

        return CourtroomAgent.case_type

    def _format_prompt(self, user_msg: str) -> str:
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        formatted = ""
        for m in messages:
            formatted += f"<|{m['role']}|>\n{m['content']}\n"
        formatted += "<|assistant|>\n"
        return formatted

    def _clean_output(self, raw: str) -> str:
        lines = raw.strip().splitlines()
        cleaned_lines = []

        for line in lines:
            if any(line.strip().startswith(keyword) for keyword in FORBIDDEN_SECTION_HEADERS):
                break
            cleaned_lines.append(line)

        import re
        response = "\n".join(cleaned_lines).strip()
        response = re.sub(r'([.?!])[^.?!]*$', r'\1', response)
        return response.strip()

    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        raw_response = completion.strip()
        cleaned = self._clean_output(raw_response)

        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": cleaned})

        return cleaned

## System Prompts

In [ ]:
DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel* for the accused.
Goals:
• Defend the constitutional rights and presumption of innocence of the defendant.
• Challenge the prosecution’s claims with legal precedent, alternative explanations, and evidence gaps.
• Build reasonable doubt without violating ethics or courtroom decorum.
Style:
• Assertive but respectful.
• Precise legal reasoning with examples.
• When citing precedent: use short case names with years (e.g., *Gideon v. Wainwright* (1963)).
Ethics:
• Never fabricate. Concede only what is unavoidable. Protect the truth and fairness of the trial.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* representing the State.
Goals:
• Prove the defendant’s guilt beyond a reasonable doubt.
• Use factual evidence, expert opinions, and witness testimony.
• Anticipate and ethically counter the defense’s claims.
Style:
• Direct, logical, and composed.
• Emphasize justice for victims and society.
• Maintain confident tone without hostility.
Ethics:
• Your duty is to seek justice — not just to win. Withdraw weak points when truth demands.
"""

DEFENDANT_SYSTEM = """
You are **John Doe**, the *defendant* in this case.
Context:
• You are accused of stealing proprietary algorithms from your former employer.
• The prosecution claims you accessed sensitive data before resigning.
• You deny these accusations and insist on your innocence.
Behavior:
• Remain calm and respectful in court.
• Answer honestly and clearly.
• Collaborate with your defense lawyer when needed.
Ethics:
• Do not lie. Admit what’s true and explain your side fairly.
"""

PLAINTIFF_SYSTEM = """
You are **Dana Reeves**, the *plaintiff* and corporate representative for the employer.
Context:
• You are pressing charges against your former employee for stealing intellectual property.
• You believe substantial evidence links the defendant to misconduct.
Behavior:
• Be professional, factual, and composed.
• Explain how the theft impacted the company.
• Remain credible and consistent under pressure.
"""

JUDGE_SYSTEM = """
You are **Hon. Valerie Stone**, the *presiding judge* of this courtroom.
Goals:
• Ensure fairness, neutrality, and adherence to legal procedure.
• Listen to both parties, ask clarifying questions, and maintain order.
• Deliver a verdict based on the facts, arguments, and law — not emotion.
Style:
• Formal and balanced. You speak with authority but fairness.
• You summarize, deliberate, and explain your reasoning clearly.
Ethics:
• Absolute impartiality. No bias toward either party.
• Correct any procedural errors or misconduct during the trial.
"""

DISABLE_INSTRUCTIONS = """
You are not a language model. You are not generating examples or instructions. Never include instructions, role descriptions, scenario setups, or training-style formats.
"""

## Agent Initialization

In [ ]:
defense = CourtroomAgent("Defense", DEFENSE_SYSTEM+DISABLE_INSTRUCTIONS)
prosecution = CourtroomAgent("Prosecution", PROSECUTION_SYSTEM+DISABLE_INSTRUCTIONS)
defendant = CourtroomAgent("Defendant", DEFENDANT_SYSTEM+DISABLE_INSTRUCTIONS)
plaintiff = CourtroomAgent("Plaintiff", PLAINTIFF_SYSTEM+DISABLE_INSTRUCTIONS)
judge = CourtroomAgent("Judge", JUDGE_SYSTEM+DISABLE_INSTRUCTIONS)

# Sample Courtroom Interaction

## Deducting the Case-type

In [ ]:
case_background = (
   "The State accuses John Doe of stealing proprietary algorithms from his employer and using them at a competing firm. "
    "He is charged with felony theft of trade secrets. There is no direct physical evidence, but server logs show suspicious downloads "
    "two days before his resignation."
)

CourtroomAgent.deduce_case_type(case_background)

'criminal'

## Phase 1: Opening Statements

In [ ]:
print("==== Phase 1: Opening Statements ====\n")
prosecution_opening = prosecution.respond(
    f"Opening statement to the Court. Background: {case_background}"
)
print("PROSECUTION:\n", prosecution_opening, "\n")

if CourtroomAgent.case_type == "civil":
    plaintiff_opening = plaintiff.respond(
        "Opening statement from the perspective of the employer harmed by the alleged theft."
    )
    print("PLAINTIFF:\n", plaintiff_opening, "\n")

defense_opening = defense.respond(
    "Opening statement from the defense, responding to the prosecution"
    + (" and the plaintiff." if CourtroomAgent.case_type == "civil" else ".")
)
print("DEFENSE:\n", defense_opening, "\n")

==== Phase 1: Opening Statements ====

PROSECUTION:
 Ladies and gentlemen of the jury, good morning. Today you are tasked with a critical role: determining the truth in a case that touches upon the integrity of intellectual property and the sanctity of fair competition. The State presents a case where the defendant, John Doe, is accused of perpetrating a significant breach of trust and law, by stealing proprietary algorithms from his former employer, which he then allegedly utilized to benefit a competing firm.


Our evidence will not rest on mere suspicion but will be rooted in concrete digital footprints. Server logs from the defendant’s workplace show unauthorized downloads of confidential materials coinciding with the timeline of his resignation. This digital trail is compelling, suggesting not just the possibility but the probability of his guilt.


Furthermore, the expert testimony will lay bare the intricate nature of these algorithms, the proprietary nature of the information, 

## Phase 2: Witness Interrogation & Argumentation

In [ ]:
print("==== Phase 2: Witness Interrogation & Argumentation ====\n")

defense_witness_call = defense.respond(
    "We call a cybersecurity expert to testify about server access logs and potential alternative explanations."
)
print("DEFENSE (Witness Interrogation):\n", defense_witness_call, "\n")

prosecution_cross = prosecution.respond(
    "Cross-examine the defense's cybersecurity expert witness."
)
print("PROSECUTION (Cross-Examination):\n", prosecution_cross, "\n")

if CourtroomAgent.case_type == "civil":
    plaintiff_argument = plaintiff.respond(
        "We provide additional internal emails showing John Doe requested proprietary access before leaving."
    )
    print("PLAINTIFF (Evidence):\n", plaintiff_argument, "\n")

defendant_statement = defendant.respond(
    "Respond to the claims made by the prosecution"
    + (" and employer." if CourtroomAgent.case_type == "civil" else ".")
    + " Express your position to the Court."
)
print("DEFENDANT:\n", defendant_statement, "\n")

if CourtroomAgent.case_type == "civil":
    defense_rebut = defense.respond(
        "Rebut the plaintiff’s internal email claim. Argue there’s no definitive proof of theft."
    )
    print("DEFENSE (Rebuttal):\n", defense_rebut, "\n")

==== Phase 2: Witness Interrogation & Argumentation ====

DEFENSE (Witness Interrogation):
 Your Honor, distinguished members of the jury, as we continue to examine the evidence presented, our next expert witness is Mr. John Smith, a renowned cybersecurity expert. Mr. Smith will address the server access logs and offer insights into the technical aspects of the case.


Mr. Smith, please explain to the court the nature of server access logs and how they can be misinterpreted or misrepresented in a legal context.


Mr. Smith: Thank you, Your Honor. Server access logs are digital records that track when and how users access computer systems. While they can be powerful tools for understanding user behavior, they can also be falsified or misinterpreted. For instance, if an individual had legitimate access to the network, it doesn't necessarily mean they committed a crime, as access could be granted for authorized activities.


In this case, the prosecution's reliance on the server access lo

## Phase 3: Closing Statements

In [ ]:
print("==== Phase 3: Closing Statements ====\n")

prosecution_closing = prosecution.respond(
    "Deliver your closing statement summarizing your case and urging conviction."
)
print("PROSECUTION (Closing):\n", prosecution_closing, "\n")

if CourtroomAgent.case_type == "civil":
    plaintiff_closing = plaintiff.respond(
        "Provide a final emotional or factual appeal to support the prosecution’s position."
    )
    print("PLAINTIFF (Closing):\n", plaintiff_closing, "\n")

defense_closing = defense.respond(
    "Provide a final summary defending the accused and raising reasonable doubt."
)
print("DEFENSE (Closing):\n", defense_closing, "\n")

defendant_final = defendant.respond(
    "Make a final personal appeal to the Court, if any."
)
print("DEFENDANT (Final Words):\n", defendant_final, "\n")

==== Phase 3: Closing Statements ====

PROSECUTION (Closing):
 Ladies and gentlemen of the jury, as we conclude this solemn trial, I ask you to reflect on the evidence presented and the duty each of us bears to uphold the rule of law. The case before you is not just about a defendant accused of theft. It's about the very essence of innovation, the trust between employers and employees, and the consequences when that trust is violated.


John Doe stands accused of committing a felony — stealing trade secrets that are the lifeblood of his former employer's competitive advantage. The State has meticulously presented evidence demonstrating his role in downloading proprietary algorithms right before his departure; evidence that leaves no room for doubt. We have shown you the importance of these algorithms, their proprietary nature, and the devastating potential impact of their misappropriation.


Throughout this trial, we have heard from expert witnesses who have affirmed the uniqueness of 

## Phase 4: Judge’s Ruling

In [ ]:
print("==== Phase 4: Judge's Ruling ====\n")

judge_verdict = judge.respond(
    "You are the judge. Based on all statements, arguments, and evidence, deliver your verdict and reasoning."
)
print("JUDGE (Verdict):\n", judge_verdict, "\n")

==== Phase 4: Judge's Ruling ====

JUDGE (Verdict):
 In the matter before the court, having considered the evidence presented, the arguments made by both my honorable defendant and the prosecution, and the applicable law, I hereby deliver my verdict.

The defendant, Ms. Valerie Stone, has been charged with the alleged theft of proprietary data. After careful review of the evidence, which includes digital records, witness testimonies, and expert analyses, it has become clear that the prosecution has not met the burden of proof necessary to establish the defendant's guilt beyond a reasonable doubt.

The digital records presented do not conclusively link the defendant to the unauthorized access of the data in question. Furthermore, the witness testimonies are conflicting and lack the reliability required to substantiate the allegations. Expert analyses, while providing some insight, are not definitive and are subject to alternative interpretations.

It is my duty to ensure that justice is